In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import cv2

output_folder="../output"

# LSD method

In [ ]:
# Let's define a function that attempts to detect the lines automatically
def automatic_line_detection(image):
    # Convert the image to grayscale if it's not already
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    
    H, W = image.shape[:2]
    lsd = cv2.createLineSegmentDetector(0)

    lines = lsd.detect(gray)[0]

    # Initialize an empty list to hold the clamped lines
    clamped_lines = []

    for line in lines:
        x1, y1, x2, y2 = line[0]

        # Clamp the coordinates
        x1 = min(max(round(x1), 0), W - 1)
        y1 = min(max(round(y1), 0), H - 1)
        x2 = min(max(round(x2), 0), W - 1)
        y2 = min(max(round(y2), 0), H - 1)

        # Append the clamped line to the list
        clamped_lines.append([x1, y1, x2, y2])

    print(f'Original lines detected: {len(lines)}')

    # Draw lines on a copy of the original image
    drawn_img = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR) if len(image.shape) == 2 else image.copy()
    if clamped_lines is not None:
        print(f"Lines detected: {len(lines)}")
        for line in clamped_lines:
            x1, y1, x2, y2 = line
            cv2.line(drawn_img, (x1, y1), (x2, y2), (0, 255, 0), 1)

    # Show the detected lines
    plt.imshow(cv2.cvtColor(drawn_img, cv2.COLOR_BGR2RGB))
    plt.title('Automatically Detected Lines')
    plt.show()

    return lines

image = cv2.imread("/Users/mlapin/Development/personal/NaturalAGI/tests/test-data/triangle_black_0.png")
# Run the automatic line detection on the new image
detected_lines_automatic = automatic_line_detection(image)

# Generate test image

In [ ]:
import os

output_folder = "../../output"
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

coef = 6
line_width = 1
new_img = np.zeros((round(500 / coef), round(500 / coef), 3), dtype=np.uint8)
# TODO Triangle
cv2.line(new_img, (0, 0), (round(300 / coef), round(300 / coef)), (225, 225, 225), line_width)
cv2.line(new_img, (round(200 / coef), round(100 / coef)), (round(100 / coef), round(400 / coef)), (225, 225, 225), line_width)
cv2.line(new_img, (round(100 / coef), round(300 / coef)), (round(500 / coef), round(100 / coef)), (225, 225, 225), line_width)
cv2.line(new_img, (0, 0), (round(300 / coef), round(300 / coef)), (225, 225, 225), line_width)

cv2.line(new_img, (39, 38), (29, 29), (225, 177, 225), line_width)
cv2.circle(new_img, (29, 29), 2, (225, 225, 177))

# TODO Square
# cv2.line(new_img, (round(100 / coef), round(80 / coef)), (round(100 / coef), round(420 / coef)), (213, 225, 163), line_width)
# cv2.line(new_img, (round(80 / coef), round(100 / coef)), (round(420 / coef), round(100 / coef)), (213, 225, 163), line_width)
# cv2.line(new_img, (round(400 / coef), round(80 / coef)), (round(400 / coef), round(420 / coef)), (189, 196, 167), line_width)
# cv2.line(new_img, (round(80 / coef), round(400 / coef)), (round(420 / coef), round(400 / coef)), (189, 196, 167), line_width)
cv2.imwrite(f"{output_folder}/triangle_10.png", new_img)
plt.imshow(cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB))
plt.title('New image')
plt.show()

Generate test triangles

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def generate_random_triangle():
    """Generates the coordinates of three random points to form a triangle."""
    return np.random.rand(3, 2)

# Set up plot
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for ax in axes:
    # Generate random triangle
    triangle = generate_random_triangle()

    # Close the triangle by repeating the first point at the end
    closed_triangle = np.vstack([triangle, triangle[0]])

    # Plot
    ax.plot(closed_triangle[:, 0], closed_triangle[:, 1], color='black', linewidth=2)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal', adjustable='box')
    ax.axis('off')

plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()

# Convert image to the gray

In [ ]:
# Converter
image_name = "triangle_10"
output_folder="../output"
image_to_export = cv2.imread(f"{output_folder}/{image_name}.png")
# Convert the image to grayscale if it's not already
if len(image_to_export.shape) == 3:
    gray = cv2.cvtColor(image_to_export, cv2.COLOR_BGR2GRAY)
else:
    gray = image
    
cv2.imwrite(f"{output_folder}/{image_name}_gray.png", gray)
cv2.imwrite(f"{output_folder}/{image_name}_comp.png", gray)

# Compress the image

In [ ]:
from PIL import Image

def compress_and_resize(image_path, output_path, new_width, new_height, quality):
    # Open the image
    with Image.open(image_path) as img:
        # Resize the image
        img = img.resize((new_width, new_height))

        # Save the image with specified quality
        # Quality is an integer between 0 (worst) to 100 (best)
        img.save(output_path, quality=quality, optimize=True)
        
compress_and_resize("../output/triangle_10_inv.png", "../output/triangle_10_comp_inv.png", 85, 85, 100)

# Hough Transform

In [ ]:
from skimage.transform import (hough_line, hough_line_peaks)
import numpy as np
import cv2
from matplotlib import pyplot as plt

image = cv2.imread(f'../tests/generated_samples/triangle_0_0_0_2.bmp', 0) #Fails if uses as-is due to bright background.
#Also try lines2 to see how it only picks up straight lines
#Invert images to show black background
# image = ~image  #Invert the image (only if it had bright background that can confuse hough)
plt.imshow(image, cmap='gray')

# Set a precision of 1 degree. (Divide into 180 data points)
# You can increase the number of points if needed. 
tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 180)

# Perform Hough Transformation to change x, y, to h, theta, dist space.
hspace, theta, dist = hough_line(image, tested_angles)

plt.figure(figsize=(10,10))
plt.imshow(hspace)  


#Now, to find the location of peaks in the hough space we can use hough_line_peaks
h, q, d = hough_line_peaks(hspace, theta, dist)


#################################################################
#Example code from skimage documentation to plot the detected lines
angle_list=[]  #Create an empty list to capture all angles

# Generating figure 1
fig, axes = plt.subplots(1, 3, figsize=(15, 6))
ax = axes.ravel()

ax[0].imshow(image, cmap='gray')
ax[0].set_title('Input image')
ax[0].set_axis_off()

ax[1].imshow(np.log(1 + hspace),
             extent=[np.rad2deg(theta[-1]), np.rad2deg(theta[0]), dist[-1], dist[0]],
             cmap='gray', aspect=1/1.5)
ax[1].set_title('Hough transform')
ax[1].set_xlabel('Angles (degrees)')
ax[1].set_ylabel('Distance (pixels)')
ax[1].axis('image')

ax[2].imshow(image, cmap='gray')

origin = np.array((0, image.shape[1]))

for _, angle, dist in zip(*hough_line_peaks(hspace, theta, dist)):
    angle_list.append(angle) #Not for plotting but later calculation of angles
    y0, y1 = (dist - origin * np.cos(angle)) / np.sin(angle)
    ax[2].plot(origin, (y0, y1), '-r')
ax[2].set_xlim(origin)
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_axis_off()
ax[2].set_title('Detected lines')

plt.tight_layout()
plt.show()

###############################################################
# Convert angles from radians to degrees (1 rad = 180/pi degrees)
angles = [a*180/np.pi for a in angle_list]

# Compute difference between the two lines
angle_difference = np.max(angles) - np.min(angles)
print(180 - angle_difference)   #Subtracting from 180 to show it as the small angle between two lines

In [ ]:
import cv2
import numpy as np

def line_intersection(line1, line2):
    """Find the intersection of two lines given in Hesse normal form."""
    rho1, theta1 = line1
    rho2, theta2 = line2
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    x0, y0 = np.linalg.solve(A, b)
    return x0, y0

def detect_lines_and_intersections(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Line detection using Hough Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

    # Find the intersection points
    intersections = []
    for i in range(len(lines)):
        for j in range(i+1, len(lines)):
            intersect = line_intersection(lines[i][0], lines[j][0])
            if intersect is not None:
                intersections.append(intersect)

    # Optional: Draw lines and intersections on the image for visualization
    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

    for intersect in intersections:
        cv2.circle(img, (int(intersect[0]), int(intersect[1])), 5, (0, 255, 0), -1)

    cv2.imwrite('output_image.png', img)
    return intersections

# Usage example
intersections = detect_lines_and_intersections(f'/Users/mlapin/Development/personal/NaturalAGI/tests/squares/square_1.png')
print(intersections)


# Probabilistic Hough

In [ ]:
from skimage.transform import probabilistic_hough_line
from skimage.feature import canny
from matplotlib import cm

# Line finding using the Probabilistic Hough Transform
image = cv2.imread(f'/Users/mlapin/Development/personal/NaturalAGI/tests/squares/square_1.png' , 0) #Fails if uses as-is due to bright background.
edges = canny(image, sigma=1.0, low_threshold=10, high_threshold=20)
lines = probabilistic_hough_line(image, threshold=10, line_length=2, line_gap=7)

print(f"Lines detected: {len(lines)}")
        
# Generating figure 2
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')

ax[1].imshow(edges, cmap=cm.gray)
ax[1].set_title('Canny edges')

ax[2].imshow(edges * 0)
for line in lines:
    p0, p1 = line
    ax[2].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[2].set_xlim((0, image.shape[1]))
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_title('Probabilistic Hough')

for a in ax:
    a.set_axis_off()

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from math import atan2, degrees

def calculate_angle(line):
    """Calculate the angle of a line segment."""
    p0, p1 = line
    return degrees(atan2(p1[1] - p0[1], p1[0] - p0[0]))

def are_angles_similar(angle1, angle2, threshold=10):
    """Check if two angles are similar within a threshold."""
    return abs(angle1 - angle2) < threshold

def cluster_lines_by_angle(lines, angle_threshold):
    """Cluster lines based on the similarity of their angles."""
    clusters = {}
    for line in lines:
        angle = calculate_angle(line)
        added_to_cluster = False
        for cluster_angle in clusters.keys():
            if are_angles_similar(angle, cluster_angle, angle_threshold):
                clusters[cluster_angle].append(line)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters[angle] = [line]
    return clusters

def midpoint(line):
    """Calculate the midpoint of a line segment."""
    return [(line[0][0] + line[1][0]) / 2, (line[0][1] + line[1][1]) / 2]

def distance(point1, point2):
    """Calculate the Euclidean distance between two points."""
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def merge_lines(lines):
    """Merge lines that are close to each other."""
    merged_lines = []
    for line in lines:
        added = False
        for merged_line in merged_lines:
            if are_angles_similar(calculate_angle(line), calculate_angle(merged_line), 10):
                if distance(midpoint(line), midpoint(merged_line)) < 20:  # Adjust this threshold as needed
                    # Merge lines by averaging their endpoints
                    new_line = [
                        ((line[0][0] + merged_line[0][0]) / 2, (line[0][1] + merged_line[0][1]) / 2),
                        ((line[1][0] + merged_line[1][0]) / 2, (line[1][1] + merged_line[1][1]) / 2)
                    ]
                    merged_lines.remove(merged_line)
                    merged_lines.append(new_line)
                    added = True
                    break
        if not added:
            merged_lines.append(line)
    return merged_lines

unioned_lines = merge_lines(lines)
print(unioned_lines)

print(f"Lines found: {len(unioned_lines)}")

# Generating figure 2
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')

ax[1].imshow(edges * 0)
for line in unioned_lines:
    p0, p1 = line
    ax[1].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[1].set_xlim((0, image.shape[1]))
ax[1].set_ylim((image.shape[0], 0))
ax[1].set_title('Probabilistic Hough')

# for a in ax:
    # a.set_axis_off()

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from math import atan2, degrees

def calculate_angle(line):
    """Calculate the angle of a line segment."""
    p0, p1 = line
    return degrees(atan2(p1[1] - p0[1], p1[0] - p0[0]))

def are_angles_similar(angle1, angle2, threshold=10):
    """Check if two angles are similar within a threshold."""
    return abs(angle1 - angle2) < threshold

def endpoint_distance(line1, line2):
    p0, p1 = line1
    p2, p3 = line2
    distances = [np.hypot(p0[0] - p2[0], p0[1] - p2[1]), 
                 np.hypot(p0[0] - p3[0], p0[1] - p3[1]),
                 np.hypot(p1[0] - p2[0], p1[1] - p2[1]), 
                 np.hypot(p1[0] - p3[0], p1[1] - p3[1])]
    return min(distances)

def cluster_lines_by_angle(lines, angle_threshold):
    """Cluster lines based on the similarity of their angles."""
    clusters = {}
    for line in lines:
        angle = calculate_angle(line)
        added_to_cluster = False
        for cluster_angle in clusters.keys():
            if are_angles_similar(angle, cluster_angle, angle_threshold):
                clusters[cluster_angle].append(line)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters[angle] = [line]
    return clusters

def midpoint(line):
    """Calculate the midpoint of a line segment."""
    return [(line[0][0] + line[1][0]) / 2, (line[0][1] + line[1][1]) / 2]

def distance(point1, point2):
    """Calculate the Euclidean distance between two points."""
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def merge_lines(lines, angle_threshold=10, distance_threshold=20):
    merged_lines = []
    while lines:
        line = lines.pop(0)
        line_angle = calculate_angle(line)
        merged = False
        for index, merged_line in enumerate(merged_lines):
            merged_line_angle = calculate_angle(merged_line)
            if are_angles_similar(line_angle, merged_line_angle, angle_threshold):
                if endpoint_distance(line, merged_line) < distance_threshold:
                    # Merge lines by averaging their endpoints
                    new_line = [
                        ((line[0][0] + merged_line[0][0]) / 2, (line[0][1] + merged_line[0][1]) / 2),
                        ((line[1][0] + merged_line[1][0]) / 2, (line[1][1] + merged_line[1][1]) / 2)
                    ]
                    merged_lines[index] = new_line
                    merged = True
                    break
        if not merged:
            merged_lines.append(line)
    return merged_lines

# Assuming 'lines' is a list of lines detected by the Hough transform
angle_threshold = 10  # Define the angle threshold for considering lines to be similar
lines_clustered_by_angle = cluster_lines_by_angle(lines, angle_threshold)

# Now merge lines within each angle cluster based on proximity
merged_lines = []
for angle, lines_in_cluster in lines_clustered_by_angle.items():
    merged_lines.extend(merge_lines(lines_in_cluster))


# Generating figure 2
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')

ax[1].imshow(edges * 0)
for line in merged_lines:
    p0, p1 = line
    ax[1].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[1].set_xlim((0, image.shape[1]))
ax[1].set_ylim((image.shape[0], 0))
ax[1].set_title('Probabilistic Hough')

# for a in ax:
    # a.set_axis_off()

plt.tight_layout()
plt.show()

In [ ]:

def calculate_slope_intercept(x1, y1, x2, y2):
    if x2 == x1:
        return None, None  # This indicates a vertical line
    slope = (y2 - y1) / (x2 - x1)
    intercept = y1 - slope * x1
    return slope, intercept

def find_intersection(slope1, intercept1, slope2, intercept2):
    if slope1 is None or slope2 is None:
        return None  # One or both lines are vertical
    if slope1 == slope2:
        return None  # Parallel lines
    x = (intercept2 - intercept1) / (slope1 - slope2)
    y = slope1 * x + intercept1
    return x, y

def all_intersections(lines):
    intersections = []
    print(f"Lines: {lines}")
    for i in range(len(lines)):
        for j in range(i + 1, len(lines)):
            line1 = lines[i]
            line2 = lines[j]

            slope1, intercept1 = calculate_slope_intercept(*line1[0], *line1[1])
            slope2, intercept2 = calculate_slope_intercept(*line2[0], *line2[1])

            if slope1 is not None and slope2 is not None:
                intersection = find_intersection(slope1, intercept1, slope2, intercept2)
                if intersection:
                    intersections.append(intersection)
            # Add handling for vertical lines if needed

    return intersections

# Find intersections
intersections = all_intersections(unioned_lines)
print(intersections)
print(f"Lines found: {len(unioned_lines)}")
print(f"INtersection points found: {len(intersections)}")

# Generating figure 2
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')

ax[1].imshow(edges * 0)
for line in unioned_lines:
    p0, p1 = line
    ax[1].plot((p0[0], p1[0]), (p0[1], p1[1]))
for point in intersections:
    ax[1].plot(point[0], point[1], 'ro')  # mark intersection
ax[1].set_xlim((0, image.shape[1]))
ax[1].set_ylim((image.shape[0], 0))
ax[1].set_title('Probabilistic Hough')

# for a in ax:
    # a.set_axis_off()

plt.tight_layout()
plt.show()


In [ ]:
# image_name_to_detect = "square"
image_name_to_detect = "triangle"
image = cv2.imread(f"/Users/mlapin/Development/personal/NaturalAGI/tests/squares/square_1.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Generating figure 2
fig, axes = plt.subplots(1, 4, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

class LineDetector:

    def detect_lines(self, image):
        # Convert the image to grayscale if it's not already
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        edges = canny(image, sigma=1.0, low_threshold=10, high_threshold=20)
        lines = probabilistic_hough_line(gray, threshold=20, line_length=2, line_gap=7)
        
        angle_threshold = 2  # Define the angle threshold for considering lines to be similar
        lines_clustered_by_angle = self.cluster_lines_by_angle(lines, angle_threshold)
        
        # Now merge lines within each angle cluster based on proximity
        final_lines = []
        for angle, lines_in_cluster in lines_clustered_by_angle.items():
            merged_lines=self.merge_lines(lines_in_cluster)
            for line in merged_lines:
                final_lines.append([line[0][0], line[0][1], line[1][0], line[1][1]])
        print(f"Merged lines: {final_lines}")
        print(f"Lines found: {len(final_lines)}")
        return lines, final_lines
    
    def calculate_angle(self, line):
        """Calculate the angle of a line segment."""
        p0, p1 = line
        return degrees(atan2(p1[1] - p0[1], p1[0] - p0[0]))


    def are_angles_similar(self, angle1, angle2, threshold=10):
        """Check if two angles are similar within a threshold."""
        return abs(angle1 - angle2) < threshold
    
    def endpoint_distance(self, line1, line2):
        p0, p1 = line1
        p2, p3 = line2
        distances = [np.hypot(p0[0] - p2[0], p0[1] - p2[1]), 
                    np.hypot(p0[0] - p3[0], p0[1] - p3[1]),
                    np.hypot(p1[0] - p2[0], p1[1] - p2[1]), 
                    np.hypot(p1[0] - p3[0], p1[1] - p3[1])]
        return min(distances)
    
    def cluster_lines_by_angle(self, lines, angle_threshold):
        """Cluster lines based on the similarity of their angles."""
        clusters = {}
        for line in lines:
            angle = calculate_angle(line)
            added_to_cluster = False
            for cluster_angle in clusters.keys():
                if self.are_angles_similar(angle, cluster_angle, angle_threshold):
                    clusters[cluster_angle].append(line)
                    added_to_cluster = True
                    break
            if not added_to_cluster:
                clusters[angle] = [line]
        return clusters

    def midpoint(self, line):
        """Calculate the midpoint of a line segment."""
        return [(line[0][0] + line[1][0]) / 2, (line[0][1] + line[1][1]) / 2]

    def distance(self, point1, point2):
        """Calculate the Euclidean distance between two points."""
        return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

    def merge_lines(self, lines, angle_threshold=2, distance_threshold=5):
        merged_lines = []
        while lines:
            line = lines.pop(0)
            line_angle = self.calculate_angle(line)
            merged = False
            for index, merged_line in enumerate(merged_lines):
                merged_line_angle = self.calculate_angle(merged_line)
                if self.are_angles_similar(line_angle, merged_line_angle, angle_threshold):
                    if self.endpoint_distance(line, merged_line) < distance_threshold:
                        print(f"Lines to merge: {line} and {merged_line}")
                        ax[3].imshow(edges * 0)
                        ax[3].plot((line[0][0], line[1][0]), (line[0][1], line[1][1]))
                        ax[3].plot((merged_line[0][0], merged_line[1][0]), (merged_line[0][1], merged_line[1][1]))
                        ax[3].set_xlim((0, image.shape[1]))
                        ax[3].set_ylim((image.shape[0], 0))
                        ax[3].set_title('Lines to merge plot')
                        # Merge lines by averaging their endpoints
                        new_line = [
                            (min(line[0][0], merged_line[0][0], line[1][0], merged_line[1][0]),
                            min(line[0][1], merged_line[0][1], line[1][1], merged_line[1][1])),
                            (max(line[0][0], merged_line[0][0], line[1][0], merged_line[1][0]),
                            max(line[0][1], merged_line[0][1], line[1][1], merged_line[1][1])),
                        ]
                        merged_lines[index] = new_line
                        merged = True
                        break
            if not merged:
                merged_lines.append(line)
        return merged_lines
    
detector = LineDetector()
pre_lines, lines = detector.detect_lines(image)
print(f"Lines detected ({len(pre_lines)}): {pre_lines}")
print(f"Lines after post-processing ({len(lines)}): {lines}")

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')

ax[1].imshow(edges * 0)
for line in pre_lines:
    print(f"Plotting pre-processed line: {line}")
    ax[1].plot((line[0][0], line[1][0]), (line[0][1], line[1][1]))
ax[1].set_xlim((0, image.shape[1]))
ax[1].set_ylim((image.shape[0], 0))
ax[1].set_title('Pre-processed lines')

ax[2].imshow(edges * 0)
for line in lines:
    print(f"Plotting line: {line}")
    ax[2].plot((line[0], line[2]), (line[1], line[3]))
ax[2].set_xlim((0, image.shape[1]))
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_title('Post-processed lines')

# for a in ax:
    # a.set_axis_off()

plt.tight_layout()
plt.show()

In [ ]:
import math


class HoughBundler:     
    def __init__(self,min_distance=5,min_angle=2):
        self.min_distance = min_distance
        self.min_angle = min_angle
    
    def get_orientation(self, line):
        orientation = math.atan2(abs((line[3] - line[1])), abs((line[2] - line[0])))
        return math.degrees(orientation)

    def check_is_line_different(self, line_1, groups, min_distance_to_merge, min_angle_to_merge):
        for group in groups:
            for line_2 in group:
                if self.get_distance(line_2, line_1) < min_distance_to_merge:
                    orientation_1 = self.get_orientation(line_1)
                    orientation_2 = self.get_orientation(line_2)
                    if abs(orientation_1 - orientation_2) < min_angle_to_merge:
                        group.append(line_1)
                        return False
        return True

    def distance_point_to_line(self, point, line):
        px, py = point
        x1, y1, x2, y2 = line

        def line_magnitude(x1, y1, x2, y2):
            line_magnitude = math.sqrt(math.pow((x2 - x1), 2) + math.pow((y2 - y1), 2))
            return line_magnitude

        lmag = line_magnitude(x1, y1, x2, y2)
        if lmag < 0.00000001:
            distance_point_to_line = 9999
            return distance_point_to_line

        u1 = (((px - x1) * (x2 - x1)) + ((py - y1) * (y2 - y1)))
        u = u1 / (lmag * lmag)

        if (u < 0.00001) or (u > 1):
            #// closest point does not fall within the line segment, take the shorter distance
            #// to an endpoint
            ix = line_magnitude(px, py, x1, y1)
            iy = line_magnitude(px, py, x2, y2)
            if ix > iy:
                distance_point_to_line = iy
            else:
                distance_point_to_line = ix
        else:
            # Intersecting point is on the line, use the formula
            ix = x1 + u * (x2 - x1)
            iy = y1 + u * (y2 - y1)
            distance_point_to_line = line_magnitude(px, py, ix, iy)

        return distance_point_to_line

    def get_distance(self, a_line, b_line):
        dist1 = self.distance_point_to_line(a_line[:2], b_line)
        dist2 = self.distance_point_to_line(a_line[2:], b_line)
        dist3 = self.distance_point_to_line(b_line[:2], a_line)
        dist4 = self.distance_point_to_line(b_line[2:], a_line)

        return min(dist1, dist2, dist3, dist4)

    def merge_lines_into_groups(self, lines):
        groups = []  # all lines groups are here
        # first line will create new group every time
        groups.append([lines[0]])
        # if line is different from existing groups, create a new group
        for line_new in lines[1:]:
            if self.check_is_line_different(line_new, groups, self.min_distance, self.min_angle):
                groups.append([line_new])

        return groups

    def merge_line_segments(self, lines):
        orientation = self.get_orientation(lines[0])
      
        if(len(lines) == 1):
            return np.block([[lines[0][:2], lines[0][2:]]])

        points = []
        for line in lines:
            points.append(line[:2])
            points.append(line[2:])
        if 45 < orientation <= 90:
            #sort by y
            points = sorted(points, key=lambda point: point[1])
        else:
            #sort by x
            points = sorted(points, key=lambda point: point[0])

        return np.block([[points[0],points[-1]]])

    def process_lines(self, lines):
        lines_horizontal  = []
        lines_vertical  = []
  
        for line_i in [l[0] for l in lines]:
            orientation = self.get_orientation(line_i)
            # if vertical
            if 45 < orientation <= 90:
                lines_vertical.append(line_i)
            else:
                lines_horizontal.append(line_i)

        lines_vertical  = sorted(lines_vertical , key=lambda line: line[1])
        lines_horizontal  = sorted(lines_horizontal , key=lambda line: line[0])
        merged_lines_all = []

        # for each cluster in vertical and horizontal lines leave only one line
        for i in [lines_horizontal, lines_vertical]:
            if len(i) > 0:
                groups = self.merge_lines_into_groups(i)
                merged_lines = []
                for group in groups:
                    merged_lines.append(self.merge_line_segments(group))
                merged_lines_all.extend(merged_lines)
                    
        return np.asarray(merged_lines_all)

In [ ]:
from PIL import Image, ImageOps

def invert_colors():
    # Open the image
    image = Image.open(f"{output_folder}/{image_name_to_detect}_gray.png")

    # Invert colors
    inverted_image = ImageOps.invert(image)

    # Save the inverted image
    inverted_image.save(f"{output_folder}/{image_name_to_detect}_inv.png")

# Example usage
invert_colors()

In [ ]:
import cv2 as cv
import numpy as np
import math
import matplotlib.pyplot as plt

# Your HoughBundler class code here

# Load your image here
image_name_to_detect = "triangle_10"
image = cv.imread("../tests/squares/square_9.png")

# Convert the image to grayscale if it's not already
if len(image.shape) == 3:
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
else:
    gray = image

# Detect lines using Hough Transform
lines = cv.HoughLinesP(gray, 1, np.pi / 180, threshold=15, lines=None, minLineLength=10, maxLineGap=1)

if lines is None:
    print('No lines detected')
else:
    # Initialize HoughBundler
    bundler = HoughBundler(min_distance=10, min_angle=10)

    # Process lines
    processed_lines = bundler.process_lines(lines)
    print(f"Processed lines:{processed_lines}")

    # Plotting
    plt.figure(figsize=(10, 10))
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))  # Convert to RGB for matplotlib
    for line in processed_lines:
        x1, y1, x2, y2 = line[0]
        plt.plot([x1, x2], [y1, y2], 'r')

    plt.title('Processed Lines')
    plt.axis('off')
    plt.show()


### Find intersections

In [ ]:
import random
import matplotlib.pyplot as plt
import cv2


def intersection(line1, line2):
    """
    Finds the intersection of two lines given in Hesse normal form.
    
    Returns closest integer pixel locations.
    See https://en.wikipedia.org/wiki/Line%E2%80%93line_intersection
    for derivation.
    """
    x1, y1, x2, y2 = line1[0]
    x3, y3, x4, y4 = line2[0]

    px = ( (x1*y2 - y1*x2)*(x3-x4) - (x1-x2)*(x3*y4 - y3*x4) ) / ( (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4) )
    py = ( (x1*y2 - y1*x2)*(y3-y4) - (y1-y2)*(x3*y4 - y3*x4) ) / ( (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4) )

    return [int(px), int(py)]

# Draw lines on the image
for line in processed_lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(image, (x1, y1), (x2, y2), (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)), 2)

# Find and draw intersections
intersections = []
for i in range(len(processed_lines)):
    for j in range(i+1, len(processed_lines)):
        pt = intersection(processed_lines[i], processed_lines[j])
        intersections.append(pt)
        cv2.circle(image, (pt[0], pt[1]), 5, (255, 0, 0), -1)

print("Intersections:", len(intersections), intersections)
print("processed_lines:", len(processed_lines))

# Display the result using matplotlib
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.title('Image with Lines and Intersections')
plt.show()
